# Run these after removing hastag if any on of the given module is missing 

In [158]:
#remove hastag if having problem with gensim
import sys

#!$sys.executable -m pip install keras
#!$sys.executable -m pip install nltk
import nltk

#!$sys.executable -m pip install gensim

#nltk.download('punkt')

#nltk.download('stopwords')

#!$sys.executable -m pip install tensorflow

  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)


# Data loading and cleaning

In [32]:
import pandas as pd
import numpy as np
import os

In [33]:
DATASET_DIR="Dataset and discription/"

In [34]:
X = pd.read_csv(os.path.join(DATASET_DIR, 'training_set_rel3.tsv'), sep='\t', encoding='ISO-8859-1')

In [35]:
X.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
X=X.dropna(axis=1)

In [37]:
X=X.drop(columns=['rater1_domain1','rater2_domain1'])


In [38]:
X

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",8
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10
4,5,1,"Dear @LOCATION1, I know having computers has a...",8
...,...,...,...,...
12971,21626,8,In most stories mothers and daughters are eit...,35
12972,21628,8,I never understood the meaning laughter is th...,32
12973,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",40
12974,21630,8,Trippin' on fen...,40


In [39]:
Y=X['domain1_score']

In [40]:
minimum_scores = np.array([-1, 2, 1, 0, 0, 0, 0, 0, 0])
maximum_scores = np.array([-1, 12, 6, 3, 3, 4, 4, 30, 60])

In [41]:
old_min = minimum_scores[X['essay_set']]
old_max = maximum_scores[X['essay_set']]
old_range = old_max - old_min 
new_range = (10 - 0)  
X['score'] = np.around((((X['domain1_score'] - old_min) * new_range) / old_range) )

X



,essay_id,essay_set,essay,domain1_score,score
0,1,1,"Dear local newspaper, I think effects computer...",8,6.0
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",9,7.0
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",7,5.0
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",10,8.0
4,5,1,"Dear @LOCATION1, I know having computers has a...",8,6.0
...,...,...,...,...,...
12971,21626,8,In most stories mothers and daughters are eit...,35,6.0
12972,21628,8,I never understood the meaning laughter is th...,32,5.0
12973,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",40,7.0
12974,21630,8,Trippin' on fen...,40,7.0


# Tokanization and vectorization functions

In [55]:
from nltk.corpus import stopwords
import nltk

import re
def wordlist(essay, remove_stopwords):
    
    essay = re.sub("[^a-zA-Z]", " ", essay)
    words = essay.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    return (words)

In [64]:
def Make_sentences(essay, remove_stopwords):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(essay.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(wordlist(raw_sentence, remove_stopwords))
    return sentences

In [147]:
def makeFeatureVec(words, model, num_features):
    
    featureVec = np.zeros((num_features),dtype="float32")
    num_words = 0.
    index2word_set = set(model.wv.index_to_key)
    for word in words:
        if word in index2word_set:
            num_words += 1
            featureVec = np.add(featureVec,model.wv[word])        
    featureVec = np.divide(featureVec,num_words)
    return featureVec

    

In [88]:
def getAvgFeatureVecs(essays, model, num_features):

    counter = 0
    essayFeatureVecs = np.zeros((len(essays),num_features),dtype="float32")
    for essay in essays:
        essayFeatureVecs[counter] = makeFeatureVec(essay, model, num_features)
        counter = counter + 1
    return essayFeatureVecs

# Defining production model

In [166]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from tensorflow.keras.models import Sequential, load_model, model_from_config
import tensorflow.keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
    model.summary()

    return model

# Example of pre processing , tokanization then vectorivation

# Example of tokanization

In [167]:
from gensim.models import Word2Vec
# define training data
example2='hi I am an example1 sentence here to explain thw working of above function ,Tiwari pakhandi hai,shayad isecay samajh me aa jayega'
example1='hi I am an example22 sentence here to explain thw working of above function ,Tiwari pakhandi hai,shayad isecay samajh me aa jayega'
example3='hi I am an example33 sentence here to explain thw working of above function ,Tiwari pakhandi hai,shayad isecay samajh me aa jayega'
s=[]
s +=Make_sentences(example1,remove_stopwords=True)
s +=Make_sentences(example2,remove_stopwords=True)
s +=Make_sentences(example3,remove_stopwords=True)
print(s)


[['hi', 'example', 'sentence', 'explain', 'thw', 'working', 'function', 'tiwari', 'pakhandi', 'hai', 'shayad', 'isecay', 'samajh', 'aa', 'jayega'], ['hi', 'example', 'sentence', 'explain', 'thw', 'working', 'function', 'tiwari', 'pakhandi', 'hai', 'shayad', 'isecay', 'samajh', 'aa', 'jayega'], ['hi', 'example', 'sentence', 'explain', 'thw', 'working', 'function', 'tiwari', 'pakhandi', 'hai', 'shayad', 'isecay', 'samajh', 'aa', 'jayega']]


# Example of vectorization

In [168]:
model = Word2Vec(s, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.index_to_key )
print(words)
# access vector for one word
print(model.wv['hi'])
# save model
model.save('model.bin')
# load model
new_model = Word2Vec.load('model.bin')
print(new_model)

Word2Vec(vocab=15, vector_size=100, alpha=0.025)
['jayega', 'aa', 'samajh', 'isecay', 'shayad', 'hai', 'pakhandi', 'tiwari', 'function', 'working', 'thw', 'explain', 'sentence', 'example', 'hi']
[ 7.7010952e-03  9.1176201e-03  1.1432337e-03 -8.3244424e-03
  8.4187957e-03 -3.7004054e-03  5.7439804e-03  4.3956689e-03
  9.6870130e-03 -9.3018962e-03  9.2124315e-03 -9.2854947e-03
 -6.9060475e-03 -9.1049811e-03 -5.5488548e-03  7.3718932e-03
  9.1690263e-03 -3.3273669e-03  3.7213692e-03 -3.6347338e-03
  7.8824461e-03  5.8715679e-03  4.1775697e-06 -3.6322384e-03
 -7.2279074e-03  4.7728908e-03  1.4499833e-03 -2.6092341e-03
  7.8357616e-03 -4.0453337e-03 -9.1553936e-03 -2.2538740e-03
  1.3046118e-04 -6.6413041e-03 -5.4882122e-03 -8.5001048e-03
  9.2311958e-03  7.4223080e-03 -2.9928688e-04  7.3693860e-03
  7.9533989e-03 -7.8221509e-04  6.6067595e-03  3.7699135e-03
  5.0775735e-03  7.2475909e-03 -4.7450820e-03 -2.1844471e-03
  8.7563385e-04  4.2404803e-03  3.2983597e-03  5.0968025e-03
  4.5875884e

# Onto the real model

In [169]:
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.metrics import cohen_kappa_score
import nltk
from gensim.models import Word2Vec
skf = StratifiedKFold(n_splits=5,shuffle=True)
count=1
results=[]
for train,test in skf.split(X,Y):
        print("\n--------Fold {}--------\n".format(count))
        X_test, X_train, y_test, y_train = X.iloc[test], X.iloc[train], Y.iloc[test], Y.iloc[train]
        trainE = X_train['essay']
        testE= X_test['essay']
        sentences=[]
        for essay in trainE:
        # Obtaining all sentences from the training essays.
        
            sentences +=Make_sentences(essay, remove_stopwords = True)
        
        #Initializing different parameters for the word2vec model to be used
    
        num_features = 300
        min_word_count = 40
        num_workers = 4
        context = 10
        downsampling = 1e-3

        print("Training Word2Vec Model...")
        
        #Initializing model fro vectorization
        
    
        #initializing model and loading parameters
    
        model = Word2Vec(sentences,vector_size=300, workers=num_workers, min_count = min_word_count, window = context)
        #avoiding normalization to not reduce the essence of some words used in context
    
        model.init_sims(replace=False)
    
        #saving model
    
        model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)
        
        print("--------------- Traning complete-------------------")
        
        print("--------------- Initiating vectorization on train set -------------------")
        
        
        #generating vectors train
    
        clean_train = []
        for essay in trainE:
            clean_train.append(wordlist(essay, remove_stopwords=True))
        trainDataVecs = getAvgFeatureVecs(clean_train, model, num_features)
    
        print("--------------completed-----------------")
    
        print("--------------- Initiating vectorization on test set -------------------")
        
        #generating vectors for test
    
        clean_test = []
        for essay in testE:
            clean_test.append(wordlist(essay, remove_stopwords=True))
        testDataVecs = getAvgFeatureVecs(clean_test, model, num_features)
        
        trainDataVecs = np.array(trainDataVecs)
        testDataVecs = np.array(testDataVecs)
        
        print("------------Complete----------------------") 
        
        print("------------Adding extra dimension to essay----------------------") 
        
        #Adding extra dimension to essay
        
        trainDataVecs = np.reshape(trainDataVecs, (trainDataVecs.shape[0], 1, trainDataVecs.shape[1]))
        
        testDataVecs = np.reshape(testDataVecs, (testDataVecs.shape[0], 1, testDataVecs.shape[1]))
        
        print("------------Complete----------------------") 
        
        print("------------Traning the Preprocessed Data----------------------") 
        
        #Traning the Preprocessed Data
        
        lstm_model = get_model()
        
        print('-----------Complete--------------')
        
        print('------------------fitting data----------------------')
        
        #fitting dataset
        checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=False,mode='auto')
        lstm_model.fit(trainDataVecs, y_train, batch_size=64, epochs=50,callbacks=[checkpoint])
        
        print('-----------Complete--------------')
        
        print('---------------Predicting test set------------------------')
        
        #predicting test
        
        y_pred = lstm_model.predict(testDataVecs)
    
        # Save any one of the 8 models.
        
        if count == 5:
             lstm_model.save('./final_lstm.h5')
            
        # Round y_pred to the nearest integer.
        y_pred = np.around(y_pred)
    
            # Evaluate the model on the evaluation metric. "Quadratic mean averaged Kappa"
        result = cohen_kappa_score(y_test.values,y_pred,weights='quadratic')
        print("Kappa Score: {}".format(result))
        results.append(result)

        count += 1


C:\Users\sneha\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"



--------Fold 1--------

Training Word2Vec Model...
--------------- Traning complete-------------------
--------------- Initiating vectorization on train set -------------------


<ipython-input-169-8d7cc0144a44>:39: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=False)


KeyboardInterrupt: 